## Import

In [ ]:
import sys
import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions

## Load CIFAR-10

In [ ]:
train, test = datasets.get_cifar10(ndim=3)

## モデル定義
ロードするモデルのクラス

In [ ]:
class MyModel(Chain):
    def __init__(self):
        super(MyModel, self).__init__()
        with self.init_scope():
            self.conv1 = L.Convolution2D(
                in_channels=3, out_channels=32, ksize=5, stride=1
            )
            self.bn1 = L.BatchNormalization(32)
            self.conv2 = L.Convolution2D(
                in_channels=32, out_channels=64, ksize=3, stride=1
            )
            self.bn2 = L.BatchNormalization(64)
            self.conv3 = L.Convolution2D(
                in_channels=64, out_channels=64, ksize=3, stride=1
            )
            self.bn3 = L.BatchNormalization(64)
            self.fc4 = L.Linear(None, 512)
            self.fc5 = L.Linear(512, 10)

    def __call__(self, x):
        h = F.relu(self.conv1(x))
        h = self.bn1(h)
        h = F.max_pooling_2d(h, 2)
        h = F.relu(self.conv2(h))
        h = self.bn2(h)
        h = F.relu(self.conv3(h))
        h = self.bn3(h)
        h = F.max_pooling_2d(h, 2)
        h = F.relu(self.fc4(h))
        if chainer.config.train:
            return self.fc5(h)
        return F.softmax(self.fc5(h))

## モデルをリストア

In [ ]:
model = L.Classifier(MyModel())
serializers.load_npz('my.model', model)

# Visualize

In [ ]:
import matplotlib.pyplot as plt
% matplotlib inline

## フィルタの可視化

In [ ]:
# single
plt.imshow(model.predictor.conv1.W[3, 0].data, cmap=plt.cm.gray, interpolation='nearest')

In [ ]:
def visualize_conv_layer_weights(weights):
    out_n, in_n, h, w = weights.shape

    fig = plt.figure(figsize=(16, 9))
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)
    for i in range(out_n):
        for j in range(in_n):
            weight = weights[i, j].data
            # ax = fig.add_subplot(out_n, in_n, in_n * i + j + 1, xticks=[], yticks=[])
            ax = fig.add_subplot(in_n, out_n, out_n * j + i + 1, xticks=[], yticks=[])
            ax.imshow(weight, cmap=plt.cm.gray, interpolation='nearest')

In [ ]:
visualize_conv_layer_weights(model.predictor.conv1.W)